# PHP 2022

## 8. Laravel

### 8.1 Tutorial

#### Basics

Create a project:

In [ ]:
! composer create-project laravel/laravel demo

Add created directory to GIT so we can track the changes:

In [ ]:
! git add demo/

Change working directory:

In [ ]:
%cd demo

Try running the application:

In [ ]:
! php artisan serve --port 8888

Go to [http://127.0.0.1:8888](http://127.0.0.1:8888) to view the page. Then stop the above cell.

See the created files:

In [ ]:
! ls -la .

See ignored files and directories:

In [ ]:
! cat .gitignore

Simulate clean checkot:

In [ ]:
! git clean -fdx .

In [ ]:
! git diff .

Restore the application:

In [ ]:
! composer install

In [ ]:
! php artisan serve --port 8888

Go to [http://127.0.0.1:8888](http://127.0.0.1:8888) to view the page. Then stop the above cell.

Add ```.env``` file:

In [ ]:
! cp .env.example .env

In [ ]:
! cat .env

In [ ]:
! php artisan serve --port 8888

Go to [http://127.0.0.1:8888](http://127.0.0.1:8888) to view the page. Then stop the above cell.

In [ ]:
! php artisan key:generate

In [ ]:
! php artisan serve --port 8888

The application should now run crrectly.

Go to [http://127.0.0.1:8888](http://127.0.0.1:8888) to view the page. Then stop the above cell.

There should be no changes in the code:

In [ ]:
! git diff .

#### Codeception

In [ ]:
! composer require codeception/codeception codeception/module-asserts codeception/module-phpbrowser --dev

In [ ]:
! vendor/bin/codecept bootstrap tmp_bootrstrap

In [ ]:
! mv tmp_bootrstrap/tests/ tests_codeception

In [ ]:
! mv tmp_bootrstrap/codeception.yml .

In [ ]:
! rm -rf tmp_bootrstrap/

Replace the configuration in [codeception.yml](../../../edit/08_laravel/01_tutorial/demo/codeception.yml) by:

```yml
namespace: Tests
support_namespace: Support
paths:
    tests: tests_codeception
    output: tests_codeception/_output
    data: tests_codeception/Support/Data
    support: tests_codeception/Support
    envs: tests_codeception/_envs
actor_suffix: Tester
extensions:
    enabled:
        - Codeception\Extension\RunFailed

```

You can safely remove the Unit and Functional test suites as we will not use them for now.

Run the tests:

In [ ]:
! vendor/bin/codecept run

Add a test:

In [ ]:
! vendor/bin/codecept generate:cest Acceptance Homepage

Add below code in [tests_codeception/Acceptance/HomepageCest.php](../../../edit/08_laravel/01_tutorial/demo/tests_codeception/Acceptance/HomepageCest.php):

```php
<?php

namespace Tests\Acceptance;

use Tests\Support\AcceptanceTester;

class HomepageCest
{
    public function tryToTest(AcceptanceTester $I)
    {
        $I->wantTo('see Laravel links on homepage');

        $I->amOnPage('/');

        $I->seeInTitle('Laravel');

        $I->seeLink("Documentation", "https://laravel.com/docs");
        $I->seeLink("Laracasts", "https://laracasts.com");
        $I->seeLink("Forge", "https://forge.laravel.com");
    }
}

```

To fix the URL in configuration open the [tests_codeception/Acceptance.suite.yml](../../../edit/08_laravel/01_tutorial/demo/tests_codeception/Acceptance.suite.yml) and change:

```yml
modules:
    enabled:
        - PhpBrowser:
            url: http://localhost/myapp
```

to

```yml
modules:
    enabled:
        - PhpBrowser:
            url: http://localhost:8888
```

Run the tests:

In [ ]:
import subprocess
subprocess.Popen(['php', 'artisan', 'serve', '--port', '8888'])

In [ ]:
! vendor/bin/codecept run

In [ ]:
! killall php8.1

Check the changes:

In [ ]:
! git diff .

In [ ]:
! git status .

In [ ]:
! git add .

#### Database

Start database:

In [ ]:
! docker run --name=mysql --net=host --rm --env MYSQL_ROOT_PASSWORD=root123 --env MYSQL_ROOT_HOST=% --env MYSQL_DATABASE=test --env MYSQL_USER=test --env MYSQL_PASSWORD=test123 -d mysql/mysql-server:8.0

In [ ]:
! while ! timeout 1 bash -c "echo > /dev/tcp/localhost/3306" 2> /dev/null; do sleep 1; done; echo "Done.";

Try migrating the databse:

In [ ]:
! php artisan migrate

Edit the ```.env``` file from command line, and replace:

```
DB_CONNECTION=mysql
DB_HOST=127.0.0.1
DB_PORT=3306
DB_DATABASE=laravel
DB_USERNAME=root
DB_PASSWORD=
```

by

```
DB_CONNECTION=mysql
DB_HOST=127.0.0.1
DB_PORT=3306
DB_DATABASE=test
DB_USERNAME=test
DB_PASSWORD=test123
```

You can also edit the ```.env.example``` to have the test database credentails set after clean checkout.

In [ ]:
! php artisan migrate

#### Codeception + Database

In [ ]:
! composer require codeception/module-db --dev

To connect to the database from tests, open the [tests_codeception/Acceptance.suite.yml](../../../edit/08_laravel/01_tutorial/demo/tests_codeception/Acceptance.suite.yml) and add

```yml
modules:
    enabled:
        - Db:
            dsn: 'mysql:host=127.0.0.1;dbname=test'
            user: 'test'
            password: 'test123'
```

Edit code in [tests_codeception/Acceptance/HomepageCest.php](../../../edit/08_laravel/01_tutorial/demo/tests_codeception/Acceptance/HomepageCest.php) and add below code at the end:

```php
$I->dontSeeInDatabase('users');
```

In [ ]:
import subprocess
subprocess.Popen(['php', 'artisan', 'serve', '--port', '8888'])

In [ ]:
! vendor/bin/codecept run

In [ ]:
! killall php8.1

Check the changes:

In [ ]:
! git diff .

In [ ]:
! git status .

In [ ]:
! git add .

#### Database dump

Dump the contents of the ```test``` database to ```dump.sql``` file:

In [ ]:
! mkdir tests_codeception/_data

In [ ]:
! echo "*" > tests_codeception/_data/.gitignore

In [ ]:
! echo "!.gitignore" >> tests_codeception/_data/.gitignore

In [ ]:
! mysqldump -h127.0.0.1 -u root --password=root123 test > tests_codeception/_data/dump.sql

Open and inspect the generated file [tests_codeception/_data/dump.sql](../../../edit/08_laravel/01_tutorial/demo/tests_codeception/_data/dump.sql).

Open the [tests_codeception/Acceptance.suite.yml](../../../edit/08_laravel/01_tutorial/demo/tests_codeception/Acceptance.suite.yml) and replace:

```yml
dsn: 'mysql:host=127.0.0.1;dbname=test'
user: 'test'
password: 'test123'
```

by:

```yml
dsn: 'mysql:host=127.0.0.1;dbname=test'
user: 'test'
password: 'test123'
populate: true
cleanup: true
dump: tests_codeception/_data/dump.sql
```

In [ ]:
import subprocess
subprocess.Popen(['php', 'artisan', 'serve', '--port', '8888'])

In [ ]:
! vendor/bin/codecept run

In [ ]:
! killall php8.1

Check the changes:

In [ ]:
! git diff .

In [ ]:
! git status .

In [ ]:
! git add .

#### Stop the database

In [ ]:
! docker container stop mysql

#### Frontend

In [ ]:
! npm install

In [ ]:
! npm run build

Check changes:

In [ ]:
! git diff .

In [ ]:
! git status .

In [ ]:
! git add .

#### Cleanup

Change working directory:

In [ ]:
%cd ..

Cleanup:

In [ ]:
! git reset demo/

In [ ]:
! git clean -fdx .